In [ ]:
from cmdstanpy import CmdStanModel
import pandas as pd
import arviz as az 
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
#Stan model
model = CmdStanModel(stan_file='ex1.stan')

# 5. genarated 1000 samples
samples = model.sample(fixed_param=True, iter_sampling=1000, iter_warmup=1, chains=1)
y_draws = samples.stan_variable('y')

# 6.Compute ratio of allergic reactions
ratios = y_draws / 50  # Since N is fixed at 50

# Histogram
plt.hist(ratios, bins=30, density=True, alpha=0.5, color='b')
plt.xlabel('Ratio of Allergic Reactions')
plt.ylabel('Density')
plt.title('Ratio of Allergic Reactions')
plt.show()

In [ ]:
data = {'N': 50, 'y': 7}  

# Stan model
model = CmdStanModel(stan_file='ex2.stan')

samples = model.sample(data=data, chains=4)

# 3.
y_pred = samples.stan_variable('y_pred')
ratios = y_pred / data['N']

# 3.Histogram
plt.hist(ratios, bins=30, density=True, alpha=0.5, color='b')
plt.xlabel('Ratio of Predicted Allergic Reactions')
plt.ylabel('Density')
plt.title('Predicted Allergic Reaction Ratios')
plt.show()

# 4. Compute the expected value and 94% density interval
summary = az.summary(samples, var_names=['p', 'y_pred'], hdi_prob=0.94)
print(summary)
expected_p = summary.loc['p', 'mean']
hdi_low, hdi_high = summary.loc['y_pred', ['hdi_3%', 'hdi_97%']] / data['N']
print(f"94% density interval of predicted ratio: [{hdi_low}, {hdi_high}]")

# 4. Compare it with expected value and 94% density interval of parameter p
print(f"Expected value of p: {expected_p}")
print(f"94% density interval of parameter p: [{summary.loc['p', 'hdi_3%']}, {summary.loc['p', 'hdi_97%']}]")

# 5. Compute the probability that ratio is lower than the average probability from traditional vaccines
average_probability_traditional = 0.1
probability_lower = np.mean(ratios < average_probability_traditional)
print(f"Probability that ratio is lower than the average from traditional vaccines: {probability_lower}")